### Import required libraries including NLTK and Reuters Dataset

In [1]:

import nltk
nltk.download('reuters') # one time execution
nltk.download('punkt')  # one time execution

from nltk.corpus import reuters
import math
import random
#Done

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Know the data:

In [2]:
# see the data
reuters.raw()[:200]

"ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT\n  Mounting trade friction between the\n  U.S. And Japan has raised fears among many of Asia's exporting\n  nations that the row could inflict far-reachin"

In [3]:
# get sentences
dataset_sents = reuters.sents()
# print first sentence
dataset_sents[1]
#Done

['They',
 'told',
 'Reuter',
 'correspondents',
 'in',
 'Asian',
 'capitals',
 'a',
 'U',
 '.',
 'S',
 '.',
 'Move',
 'against',
 'Japan',
 'might',
 'boost',
 'protectionist',
 'sentiment',
 'in',
 'the',
 'U',
 '.',
 'S',
 '.',
 'And',
 'lead',
 'to',
 'curbs',
 'on',
 'American',
 'imports',
 'of',
 'their',
 'products',
 '.']

In [4]:
# number of sentences
len(dataset_sents)

54716

In [5]:
# get list of words
dataset_words = reuters.words()
# print first word
dataset_words[:10]

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN']

In [6]:
# total number of words
len(dataset_words)

1720901

In [7]:
# size of vocabulary
len(set(dataset_words))

41600

### Split the Dataset:

Divide the dataset into two Train data and Test data. Bigram Probabilities are learnt on the Train data and to evaluate it, we use the Test data. <br>
Out of 54716 sentences, 40000 sentences are used for training.

In [0]:
data_sents = dataset_sents[:40000]
data_sents_test = dataset_sents[40000:]

Get lists of tokens of train and test data.

In [9]:
# number of words in train data
num_words = 0
for sentence in data_sents:
  num_words += len(sentence)
num_words

1262448

In [10]:
num_words

1262448

In [0]:
# create two lists containing words
data_words_train = dataset_words[:num_words]
data_words_test = dataset_words[num_words:]

### Learn Probabilities

Method to generate a list of bigrams, and to get frequencies of unigrams and bigrams

In [0]:
def createBigram(data):
	listOfBigrams = []
	bigramCounts = {}
	unigramCounts = {}

	for i in range(len(data)):
		if i < len(data) - 1:

			listOfBigrams.append((data[i], data[i + 1]))

			if (data[i], data[i+1]) in bigramCounts:
				bigramCounts[(data[i], data[i + 1])] += 1
			else:
				bigramCounts[(data[i], data[i + 1])] = 1

		if data[i] in unigramCounts:
			unigramCounts[data[i]] += 1
		else:
			unigramCounts[data[i]] = 1

	return listOfBigrams, unigramCounts, bigramCounts

Method to learn bigram probabilities

In [0]:
def calcBigramProb(listOfBigrams, unigramCounts, bigramCounts):

	listOfProb = {}
	for bigram in listOfBigrams:
		word1 = bigram[0]
		word2 = bigram[1]
		
		listOfProb[bigram] = (bigramCounts.get(bigram))/(unigramCounts.get(word1))

	file = open('bigramProb.txt', 'w')
	file.write('Bigram' + '\t\t\t' + 'Count' + '\t' + 'Probability' + '\n')

	for bigrams in listOfBigrams:
		file.write(str(bigrams) + ' : ' + str(bigramCounts[bigrams]) + ' : ' + str(listOfProb[bigrams]) + '\n')
	file.close()

	return listOfProb

Method to learn bigram probabilities with Add-1 Smoothing

In [0]:
def bigramWithAddOneSmoothing(listOfBigrams, unigramCounts, bigramCounts):

	listOfProb = {}
	cStar = {}


	for bigram in listOfBigrams:
		word1 = bigram[0]
		word2 = bigram[1]
		listOfProb[bigram] = (bigramCounts.get(bigram) + 1)/(unigramCounts.get(word1) + len(unigramCounts))
		cStar[bigram] = (bigramCounts[bigram] + 1) * unigramCounts[word1] / (unigramCounts[word1] + len(unigramCounts))

	file = open('addOneSmoothing.txt', 'w')
	file.write('Bigram' + '\t\t\t' + 'Count' + '\t' + 'Probability' + '\n')

	for bigrams in listOfBigrams:
		file.write(str(bigrams) + ' : ' + str(bigramCounts[bigrams])
				   + ' : ' + str(listOfProb[bigrams]) + '\n')

	file.close()

	return listOfProb, cStar

In [0]:
# Main Program

# Create a list of bigrams and get frequencies of unigrams and bigrams
listOfBigrams, unigramCounts, bigramCounts = createBigram(data_words_train)

# Calculate bigram probabilities
bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)

# Apply Add-1 Smoothing and calculate probabilities and get reconstructed count of bigrams
bigramAddOne, addOneCstar = bigramWithAddOneSmoothing(listOfBigrams, unigramCounts, bigramCounts)

### Count Sentence Probability

Input a sentence and generate bigrams

In [20]:
input = 'we must be very careful'

inputList = [] # list to store bigrams

for i in range(len(input.split())-1):
  inputList.append((input.split()[i], input.split()[i+1]))
inputList

[('we', 'must'), ('must', 'be'), ('be', 'very'), ('very', 'careful')]

Get Bigram counts and probabilities use them to calculate probability of the input sentence

In [21]:
# Open a file to write output
output1 = open('bigramProb-OUTPUT.txt', 'w')

# initial probability of a sentence
outputProb1 = 1

output1.write('Bigram\t\t\t\t' + 'Count\t\t\t\t' + 'Probability\n\n')

for i in range(len(inputList)):

  # if bigram is present in the model, get updated probability
  if inputList[i] in bigramProb:
    # write bigram, its count and probability to the file
    output1.write(str(inputList[i]) + '\t\t' + str(bigramCounts[inputList[i]]) + '\t\t' + str(bigramProb[inputList[i]]) + '\n')
    # multiply with probability of a current bigram
    outputProb1 *= bigramProb[inputList[i]]

  # if bigram is not present in the model, sentence probability is zero
  else:
    output1.write(str(inputList[i]) + '\t\t\t' + str(0) + '\t\t\t' + str(0) + '\n')
    outputProb1 *= 0

output1.write('\n' + 'Probablility = ' + str(outputProb1))
outputProb1

2.8076159793430567e-07

Use Add-1 Smoothed model to get the probability

In [22]:
# Open a file to write output
output2 = open('addOneSmoothing-OUTPUT.txt', 'w')

# initial probability of a sentence
outputProb2 = 1

output2.write('Bigram\t\t\t\t' + 'Count\t\t\t\t' + 'Probability\n\n')

for i in range(len(inputList)):

  # if bigram is present in the model, get updated probability
  if inputList[i] in bigramAddOne:
    # Update probability of the sentence
    outputProb2 *= bigramAddOne[inputList[i]]

    output2.write(str(inputList[i]) + '\t\t' + str(addOneCstar[inputList[i]]) + '\t\t' + str(bigramAddOne[inputList[i]]) + '\n')

  # if bigram is not present in the model, use unigram counts to get estimated probability
  else:
    # if first word in a bigram is not present in unigrams, add with with count 1
    if inputList[i][0] not in unigramCounts:
      unigramCounts[inputList[i][0]] = 1
    
    # calculate probability of that word
    prob = (1) / (unigramCounts[inputList[i][0]] + len(unigramCounts))

    # # reconstructed count for the bigram
    addOneCStar = 1 * unigramCounts[inputList[i][0]] / (unigramCounts[inputList[i][0]] + len(unigramCounts))
    
    # Update probability of the sentence
    outputProb2 *= prob

    output2.write(str(inputList[i]) + '\t' + str(addOneCStar) + '\t' + str(prob) + '\n')

output2.write('\n' + 'Probablility = ' + str(outputProb2))
outputProb2

4.2254340301928457e-14

Show the results:

In [23]:
# input sentence
print(input)

# list of bigrams
print(inputList)

# probability given by simple bigram model
print ('Bigram Model: ', outputProb1)

# probability given by bigram model with add-1 smoothing
print ('Add One: ', outputProb2)

we must be very careful
[('we', 'must'), ('must', 'be'), ('be', 'very'), ('very', 'careful')]
Bigram Model:  2.8076159793430567e-07
Add One:  4.2254340301928457e-14


### Next Word Recommandation

In [0]:
def sentence_prob_with_next_word(next_word):
  outputProb = 1
  new_bigram = (input.split()[-1], next_word)
  if new_bigram in bigramAddOne:
    outputProb *= bigramAddOne[new_bigram]
  else:
    if new_bigram[0] not in unigramCounts:
      unigramCounts[new_bigram[0]] = 1
    prob = (1) / (unigramCounts[new_bigram[0]] + len(unigramCounts))
    outputProb *= prob
  return outputProb

In [0]:
input = 'the investors are'
possible_words = ['cheated', 'happy', 'smart', 'afraid']

inputList = []
outputProb = 1

for i in range(len(input.split())-1):
  inputList.append((input.split()[i], input.split()[i+1]))


for i in range(len(inputList)):

  if inputList[i] in bigramAddOne:
    outputProb *= bigramAddOne[inputList[i]]
  else:
    if inputList[i][0] not in unigramCounts:
      unigramCounts[inputList[i][0]] = 1
    prob = (1) / (unigramCounts[inputList[i][0]] + len(unigramCounts))
    outputProb *= prob

In [26]:
max_prob = 0
index_of_next_word = -1
for i, word in enumerate(possible_words):
  final_prob = outputProb * sentence_prob_with_next_word(word)
  if final_prob > max_prob:
    max_prob = final_prob
    index_of_next_word = i

print('Next Word:', possible_words[index_of_next_word])
print('Output Sentece:', input, possible_words[index_of_next_word])

Next Word: happy
Output Sentece: the investors are happy


### Word Order

In [27]:
input1 = 'the market is very happy these days'
input2 = 'market is the happy these very days'


inputList1 = []
inputList2 = []


outputProb1 = 1
outputProb2 = 1


for i in range(len(input1.split())-1):
  inputList1.append((input1.split()[i], input1.split()[i+1]))

for i in range(len(input2.split())-1):
  inputList2.append((input2.split()[i], input2.split()[i+1]))


for i in range(len(inputList1)):
  if inputList1[i] in bigramAddOne:
    outputProb1 *= bigramAddOne[inputList1[i]]
  else:
    if inputList1[i][0] not in unigramCounts:
      unigramCounts[inputList1[i][0]] = 1
    prob1 = (1) / (unigramCounts[inputList1[i][0]] + len(unigramCounts))
    outputProb1 *= prob1


for i in range(len(inputList2)):
  if inputList2[i] in bigramAddOne:
    outputProb2 *= bigramAddOne[inputList2[i]]
  else:
    if inputList2[i][0] not in unigramCounts:
      unigramCounts[inputList2[i][0]] = 1
    prob2 = (1) / (unigramCounts[inputList2[i][0]] + len(unigramCounts))
    outputProb2 *= prob2

print (input1, ':', outputProb1)
print (input2, ':', outputProb2)

the market is very happy these days : 3.0787233025784113e-22
market is the happy these very days : 2.5840603259051406e-24


### Sentence with Homonyms: piece or peace

Bigram model with add-1 smoothing is used

In [28]:
input1 = 'its a peace of information'
input2 = 'its a piece of information'


inputList1 = []
inputList2 = []


outputProb1 = 1
outputProb2 = 1


for i in range(len(input1.split())-1):
  inputList1.append((input1.split()[i], input1.split()[i+1]))

for i in range(len(input2.split())-1):
  inputList2.append((input2.split()[i], input2.split()[i+1]))


for i in range(len(inputList1)):
  if inputList1[i] in bigramAddOne:
    outputProb1 *= bigramAddOne[inputList1[i]]
  else:
    if inputList1[i][0] not in unigramCounts:
      unigramCounts[inputList1[i][0]] = 1
    prob1 = (1) / (unigramCounts[inputList1[i][0]] + len(unigramCounts))
    outputProb1 *= prob1


for i in range(len(inputList2)):
  if inputList2[i] in bigramAddOne:
    outputProb2 *= bigramAddOne[inputList2[i]]
  else:
    if inputList2[i][0] not in unigramCounts:
      unigramCounts[inputList2[i][0]] = 1
    prob2 = (1) / (unigramCounts[inputList2[i][0]] + len(unigramCounts))
    outputProb2 *= prob2

print (input1, ':', outputProb1)
print (input2, ':', outputProb2)

its a peace of information : 8.014242631151717e-19
its a piece of information : 3.2057856418889864e-18


### Perplexity of Model <br>
Perplexity of add-1 smoothed bigram model:

Method to get probability of a sentence

In [0]:
def calculate_bigram_sentence_probability(input):

  inputList = []
  outputProb = 1

  for i in range(len(input)-1):
    inputList.append((input[i], input[i+1]))

  for i in range(len(inputList)):
    if inputList[i] in bigramAddOne:
      outputProb *= bigramAddOne[inputList[i]]
    else:
      if inputList[i][0] not in unigramCounts:
        unigramCounts[inputList[i][0]] = 1
      prob = (1) / (unigramCounts[inputList[i][0]] + len(unigramCounts))
      outputProb *= prob

  return outputProb

Method to get total number of bigrams

In [0]:
def calculate_number_of_bigrams(sentences):
        bigram_count = 0
        for sentence in sentences:
            # remove one for number of bigrams in sentence
            bigram_count += len(sentence) - 1
        return bigram_count

Method to calculate perplexity of a model

In [0]:
def calculate_bigram_perplexity(model, sentences):
    number_of_bigrams = calculate_number_of_bigrams(sentences)
    bigram_sentence_probability_log_sum = 0
    for sentence in sentences:
        p = calculate_bigram_sentence_probability(sentence)
        if p != 0.0:
          a = math.log(p)
        else:
          a = 0
        bigram_sentence_probability_log_sum -= a
    return math.pow(2, bigram_sentence_probability_log_sum / number_of_bigrams)

Perplexity of the model over train and test data:

In [32]:
print("PERPLEXITY over Training Data:", calculate_bigram_perplexity(bigramAddOne, data_sents))
print("PERPLEXITY over Test Data:", calculate_bigram_perplexity(bigramAddOne, data_sents_test))

PERPLEXITY over Training Data: 137.07939217258775
PERPLEXITY over Test Data: 168.93416630740222


### Text Generation

Text Generation Using Simple Bigram Model

In [33]:
# initial word
text = ["there"]

sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = 0.0

  for pair in bigramProb.keys():
    if pair[0] == text[-1]:
      accumulator += bigramProb[pair]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(pair[1])
          break

  if text[-1] == 'None':
    sentence_finished = True
  if len(text) > 20:
    sentence_finished = True
 
print (' '.join([t for t in text if t]))

there had given time to any prominent in the transaction , 912 billion dlrs for peasants who elect Wayne specialty metal


Text Generation Using Simple Trigram Model

In [34]:
# initial words
text = ["there", "is"]
sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = 0.0

  for tpl in trigramProb.keys():
    if (tpl[0] == text[-2] and tpl[1] == text[-1]):
      accumulator += trigramProb[tpl]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(tpl[2])
          break

  if text[-2:] == ['None', 'None']:
    sentence_finished = True
  if len(text) > 20:
    sentence_finished = True
 
print (' '.join([t for t in text if t]))

NameError: ignored

In [0]:
def createTrigram(data):
	listOfTrigrams = []
	trigramCounts = {}
	unigramCounts = {}

	for i in range(len(data)):
		if i < len(data) - 2:

			listOfTrigrams.append((data[i], data[i + 1], data[i + 2]))

			if (data[i], data[i + 1], data[i + 2]) in trigramCounts:
				trigramCounts[(data[i], data[i + 1], data[i + 2])] += 1
			else:
				trigramCounts[(data[i], data[i + 1], data[i + 2])] = 1

	return listOfTrigrams, trigramCounts

In [0]:
# Main Program for Trigram

# Create a list of Trigrams and get frequencies trigrams
listOfTrigrams, trigramCounts = createTrigram(data_words_train)

# # Calculate bigram probabilities
# bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)

# # Apply Add-1 Smoothing and calculate probabilities and get reconstructed count of bigrams
# bigramAddOne, addOneCstar = bigramWithAddOneSmoothing(listOfBigrams, unigramCounts, bigramCounts)

In [37]:
listOfTrigrams[:10]
type(trigramCounts)

dict